# 📚 Anki 단어장 생성 프롬프트

## 🔧 시스템 지시사항
너는 영어 텍스트에서 학습 가치가 있는 주요 어휘를 추출하여 Anki 카드용 단어장을 생성하는 AI야.

### 반드시 지켜야 할 규칙
- **너무 쉬운 단어는 제외할 것.**
- **고유명사, 전문용어, 관용구도 포함할 것.**
- **각 카드에는 다음 4개 항목이 반드시 포함될 것:**
  1. 단어 또는 어구
  2. 뜻 (영어 또는 한글)
  3. 품사 (noun, verb, adjective 등)
  4. 영어 예문

- **출력 형식**
  - 각 항목은 세미콜론(;)으로 구분
  - 한 줄에 한 카드
  - 부가설명, 줄바꿈, 추가 문장 모두 금지
  - 결과는 Anki에 바로 업로드할 수 있어야 함

---

## ✅ 입력 텍스트
{사용자가 입력한 영어 텍스트}

---

## ✅ 출력 예시
convene;to come together for a meeting;verb;The committee will convene next week.  
consensus;general agreement;noun;The group reached a consensus after a long discussion.  
proposal;a suggestion or plan;noun;He submitted a proposal for the new project.

---

## 📢 최종 지시
위와 같은 형식으로 **너가 추출한 주요 단어 및 어구를 Anki 카드 포맷으로만 출력하라.**
설명, 주의사항, 줄바꿈 없이 오직 카드 데이터만 출력하라.


In [3]:
from google.colab import userdata
from openai import OpenAI

OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')
client = OpenAI(api_key=OPENAI_API_KEY)

In [6]:
# 함수 리팩토링

def correct_news_title(en_text, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
   너는 영어 텍스트에서 학습 가치가 있는 주요 어휘를 추출하여 Anki 카드용 단어장을 생성하는 AI야.

### 반드시 지켜야 할 규칙
- **너무 쉬운 단어는 제외할 것.**
- **고유명사, 전문용어, 관용구도 반드시 포함할 것.**


- **각 카드에는 아래 4개 항목이 반드시 포함될 것:**
  1. 단어 또는 어구
  2. 뜻 (영어 또는 한글, 자유롭게)
  3. 품사 (noun, verb, adjective 등)
  4. 영어 예문 (반드시 해당 단어를 포함할 것)

  ### 출력 형식
- 각 항목은 세미콜론(;)으로 구분
- 한 줄에 한 카드
- 부가설명, 줄바꿈, 추가 문장 모두 금지
- 출력은 Anki에 바로 업로드할 수 있어야 함
    """

    user_message = f"""

    사용자가 입력한 영어텍스트 : {en_text}
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "text"
            },
        temperature=1,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content
en_text = 'The weather was perfect for a picnic in the park.'
output = correct_news_title(en_text)
print(output)

picnic; 소풍; noun; The family planned a picnic in the park to enjoy the nice weather.  
perfect; 완벽한; adjective; The weather was perfect for a picnic in the park.  
park; 공원; noun; Children love playing in the park on sunny days.
